In [10]:
# -*- coding: utf-8 -*-
 
from numpy import *
from functools import reduce  
 
#侮辱性词汇标识
AbClass = 1
 
 
def loadDataSet():
    """加载数据集合及其对应的分类"""
    wordsList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return wordsList, classVec
 
 

def doc2VecList(docList):
    # 从第一个和第二个集合开始进行并集操作，最后返回一个不重复的并集
    a = list(reduce(lambda x, y: set(x) | set(y), docList))
    #a|b返回结果为两个集合a和b的不重复并集
    return a
 

    
def words2Vec(vecList, inputWords):
    """把单子转化为词向量"""
    # 转化成一维数组
    resultVec = [0] * len(vecList)
    for word in inputWords:
        if word in vecList:
            # 在单词出现的位置上的计数加1
            resultVec[vecList.index(word)] += 1
        else:
            print('没有发现此单词')
 
    return array(resultVec)
 
 
def trainNB(trainMatrix, trainClass):
    """计算，生成每个词对于类别上的概率"""
    
    # 类别行数
    numTrainClass = len(trainClass)
    # 列数
    numWords = len(trainMatrix[0])
 
    # 全部都初始化为1， 防止出现概率为0的情况出现
    # 见于韩家炜的数据挖掘概念与技术上的讲解，避免出现概率为0的状况，影响计算，因为在数量很大的情况下，在分子和分母同时+1的情况不会
    # 影响主要的数据
    p0Num = ones(numWords)
    p1Num = ones(numWords)
    
    # 相应的单词初始化为2
    # 为了分子分母同时都加上某个数λ
    p0Words = 2.0
    p1Words = 2.0
    
    # 统计每个分类的词的总数
    # 训练数据集的行数作为遍历的条件，从1开始
    # 如果当前类别为1，那么p1Num会加上当前单词矩阵行数据，依次遍历
    # 如果当前类别为0，那么p0Num会加上当前单词矩阵行数据，依次遍历
    # 同时统计当前类别下单词的个数和p1Words和p0Words
    for i in range(numTrainClass):
        if trainClass[i] == 1:
            # 数组在对应的位置上相加
            p1Num += trainMatrix[i]
            p1Words += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Words += sum(trainMatrix[i])
    
    # 计算每种类型里面， 每个单词出现的概率
    p0Vec = log(p0Num / p0Words)
    p1Vec = log(p1Num / p1Words)
    
    # 计算在类别中1出现的概率，0类出现的概率可通过1-p得到
    pClass1 = sum(trainClass) / float(numTrainClass)
    
    return p0Vec, p1Vec, pClass1
 
 
#朴素贝叶斯分类函数, max(p0， p1)作为推断的分类
def classifyNB(testVec, p0Vec, p1Vec, pClass1):
    p1 = sum(testVec * p1Vec) + log(pClass1)      #此处计算出的p1是用对数表示，因为对数也是单调递增的
    p0 = sum(testVec * p0Vec) + log(1 - pClass1)
    if p0 > p1:
        return 0
    return 1
 
 
def printClass(words, testClass):
    if testClass == AbClass:
        print(words, '推测为：侮辱性词汇')
    else:
        print(words, '推测为：非侮辱性词汇')
 

#测试函数
def tNB():
    # 从训练数据集中提取出属性矩阵和分类数据
    docList, classVec = loadDataSet()
    
    # 生成包含所有单词的list
    # 此处生成的单词向量是不重复的
    allWordsVec = doc2VecList(docList)
    
    # 构建词向量矩阵
    # 计算docList数据集中每一行每个单词出现的次数，其中返回的trainMat是一个数组的数组
    trainMat = list(map(lambda x: words2Vec(allWordsVec, x), docList))
    
    # 训练计算每个词在分类上的概率, p0V:每个单词在非分类出现的概率， p1V:每个单词在是分类出现的概率
    # 其中概率是以ln进行计算的
    # pClass1为类别中是1的概率
    p0V, p1V, pClass1 = trainNB(trainMat, classVec)
    
    # 测试数据集
    testWords = ['my', 'cute', 'love', 'dog']
    
    # 转换成单词向量，32个单词构成的数组，如果此单词在数组中，数组的项值置1
    testVec = words2Vec(allWordsVec, testWords)
    
    # 通过将单词向量testVec代入，根据贝叶斯公式，比较各个类别的后验概率，判断当前数据的分类情况
    testClass = classifyNB(testVec, p0V, p1V, pClass1)
    
    # 打印出测试结果
    printClass(testWords, testClass)
    
    #测试数据集
    testWords = ['stop', 'stupid', 'food']
    
    # 转换成单词向量，32个单词构成的数组，如果此单词在数组中，数组的项值置1
    testVec = words2Vec(allWordsVec, testWords)
    
    # 通过将单词向量testVec代入，根据贝叶斯公式，比较各个类别的后验概率，判断当前数据的分类情况
    testClass = classifyNB(testVec, p0V, p1V, pClass1)
    
    # 打印出测试结果
    printClass(testWords, testClass)
 
 
if __name__ == '__main__':
    tNB()

['my', 'cute', 'love', 'dog'] 推测为：非侮辱性词汇
['stop', 'stupid', 'food'] 推测为：侮辱性词汇
